# NMC/C Halfcell Data

This schematic figure shows the transferability of cell data onto the vehicle level. \
Also DVA is possible on the vehicle level showing cell level features. \
However, vehicle level peculiarities must be accounted for, e.g. varying cell capacities (see stage 2) or data quality constraints (features are less pronounced).

In [ ]:
import os
import sys
import math
sys.path.append(os.path.join(os.getcwd().partition('nature_soh')[0], "nature_soh"))
from src.config_base import GeneralConfig

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator, FixedLocator
import matplotlib.patches as patches

In [ ]:
from src.data.read_feather import ReadFeather
from src.filtering.filter_methods import FilterMethods
from src.voltage_capacity_analysis.ica import ICA
from src.voltage_capacity_analysis.dva import DVA
from src.visualization.config_visualization import VisualizationConfig, instantiate_matplotlib_parameters, cm2inch
instantiate_matplotlib_parameters()
from src.visualization.colormaps import ColorMaps
from scipy import interpolate
from scipy.optimize import fsolve
from scipy.optimize import curve_fit

In [ ]:
tum_orange_cmap = ColorMaps.orange_tum()
orange_colors = tum_orange_cmap(np.linspace(0,1, 10))
tum_orange_cmap

In [ ]:
tum_blue_cmap = ColorMaps.blue_tum()
blue_colors = tum_blue_cmap(np.linspace(0,1, 10))
tum_blue_cmap

In [ ]:
tum_cmap = ColorMaps.blue_orange_tum(grayscale=1)
colors = tum_cmap(np.linspace(0,1, 5))
tum_cmap

## Read Cell File

In [ ]:
path2files = os.path.join(GeneralConfig.path2data.value,"VW","cell")

In [ ]:
def filter_func_preprocess_U(signal):
    perc_filter = 0.1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter * len(signal)))
    return filtered_signal

In [ ]:
def filter_func_preprocess_Q(signal):
    perc_filter = 0.1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return filtered_signal

In [ ]:
read_feather = ReadFeather()
read_feather.set_filter_U(filter_func_preprocess_U)
read_feather.set_filter_Q(filter_func_preprocess_Q)

In [ ]:
df_A =  read_feather.read(os.path.join(path2files,"VW_LG_78Ah_NMC_20deg_C50_Anode.feather"))
df_A = df_A[df_A["Q"]<=0.00805] # limit to above 0V 

In [ ]:
df_C_ch =  read_feather.read(os.path.join(path2files,"VW_LG_78Ah_NMC_20deg_C50_Cathode_ch.feather"))
df_C_dch =  read_feather.read(os.path.join(path2files,"VW_LG_78Ah_NMC_20deg_C50_Cathode_dch.feather"))

In [ ]:
def filter_func(signal):
    perc_filter =0.5 #% filter
    filtered_signal = FilterMethods().rolling_mean_numpy(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    filtered_signal_reverse = FilterMethods().rolling_mean_numpy(np.flip(signal),window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    #filtered_signal_reverse = FilterMethods().savgol(np.flip(signal),window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return (filtered_signal+np.flip(filtered_signal_reverse))/2

In [ ]:
df =  read_feather.read(os.path.join(path2files,"VW_LG_78Ah_NMC_20deg_CC_C50.feather"))

In [ ]:
ica_extractor = ICA()
ica_extractor.set_filter(filter_func)
ica,volt =ica_extractor.get_ica(time=df["time_h"], capacity=df["Q"], voltage=df["U"],crop_volt_V=(2.6,4.2))

In [ ]:
n=len(df_A)

In [ ]:
df_A = df_A.sample(n=n).sort_values(by=['time_h'])
df_C_ch = df_C_ch.sample(n=n).sort_values(by=['time_h'])
df_C_dch = df_C_dch.sample(n=n).sort_values(by=['time_h'])
df = df.sample(n=n).sort_values(by=['time_h'])

### Cell DVA/ICA

In [ ]:
def filter_func(signal):
    perc_filter = 0.5 #% filter
    filtered_signal = FilterMethods().rolling_mean_numpy(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    filtered_signal_reverse = FilterMethods().rolling_mean_numpy(np.flip(signal),window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return (filtered_signal+np.flip(filtered_signal_reverse))/2

In [ ]:
def filter_func_cat(signal):
    perc_filter = 2.5 #% filter
    filtered_signal = FilterMethods().rolling_mean_numpy(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    filtered_signal_reverse = FilterMethods().rolling_mean_numpy(np.flip(signal),window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return (filtered_signal+np.flip(filtered_signal_reverse))/2

In [ ]:
dva_extractor = DVA()
dva_extractor.set_filter(filter_func)
dva_A, cap_A =dva_extractor.get_dva(time=df["time_h"], capacity=df["Q"], voltage=df_A["U"],crop_cap_Ah=(0,100))
dva, cap =dva_extractor.get_dva(time=df["time_h"], capacity=df["Q"], voltage=df["U"],crop_cap_Ah=(0,100))
dva_extractor.set_filter(filter_func_cat)
dva_C_ch, cap_C_ch =dva_extractor.get_dva(time=df["time_h"], capacity=df["Q"], voltage=df_C_ch["U"],crop_cap_Ah=(0,100))
dva_C_dch, cap_C_dch =dva_extractor.get_dva(time=df["time_h"], capacity=df["Q"], voltage=df_C_dch["U"],crop_cap_Ah=(0,100))

In [ ]:
plt.plot(cap,dva,color = "black")
plt.plot(cap_C_ch-1.7,dva_C_ch-0.0007,color = "red")
plt.plot(cap_C_dch-1.7,dva_C_dch-0.0007,color = "orange")
plt.plot(cap_A*0.93+1,-1*dva_A,color = "blue")
plt.ylim([0,0.05])

This halfcell plot is schematic and should show the peaks as clear as possible
- the M and H2 Phase match best with the full cell level in the discharge curve, but the H1 peak is not visible
- the H1 peak is visible best in the charge curve

--> For the sake of clearness these two curves are merged to one, so the peaks are visible the best

stretch signals to match by hand

In [ ]:
dva_A_trafo = -1*dva_A+0.0002
cap_A_trafo = cap_A*0.93+1
cap_A_trafo[cap_A_trafo>40]*=1.011
cap_A_trafo[cap_A_trafo>60]*=1.11

In [ ]:
plt.plot(cap,dva,color = "black")
plt.plot(cap_A_trafo,dva_A_trafo,color = "blue")
plt.ylim([0,0.05])

In [ ]:
dva_C_trafo_ch = dva_C_ch-0.0014
cap_C_trafo_ch = (cap_C_ch*1.2)-9
#cap_C_trafo[cap_C_trafo>20]*=1.1

In [ ]:
dva_C_trafo_dch = dva_C_dch-0.0010
cap_C_trafo_dch = (cap_C_dch-6.4)*1.1

In [ ]:
plt.plot(cap,dva,color = "black")
plt.plot(cap_C_trafo_ch,dva_C_trafo_ch,color = "red")
plt.plot(cap_C_trafo_dch,dva_C_trafo_dch,color = "blue")

idx = np.argwhere(np.diff(np.sign(dva_C_trafo_ch - dva_C_trafo_dch))).flatten()
plt.plot(cap_C_trafo_ch.values[idx], dva_C_trafo_ch[idx], 'ro')

idx_fit = idx[1]
cap_C_trafo = np.concatenate((cap_C_trafo_ch.values[0:idx_fit], cap_C_trafo_dch.values[idx_fit:]), axis=None)
dva_C_trafo = np.concatenate((dva_C_trafo_ch[0:idx_fit], dva_C_trafo_dch[idx_fit:]), axis=None)
plt.plot(cap_C_trafo,dva_C_trafo,color = "green")
plt.ylim([0,0.05])

In [ ]:
df_C_ch_temp = df_C_ch[0:idx_fit]
df_C_dch_temp = df_C_dch[idx_fit:]
df_C_ch_temp["U"]+= df_C_dch_temp["U"].min()-df_C_ch_temp["U"].max() # match voltage values

In [ ]:
df_C = pd.concat([df_C_ch_temp, df_C_dch_temp], ignore_index=True)

## Read vehicle file

In [ ]:
def filter_func_preprocess(signal):
    perc_filter = 1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter * len(signal)))
    return filtered_signal

In [ ]:
read_feather = ReadFeather()
read_feather.set_filter_U(filter_func_preprocess)
read_feather.set_filter_Q(filter_func_preprocess)

In [ ]:
df_vw =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2021.feather"))

### DVA

In [ ]:
dva_extractor = DVA()
dva_extractor.set_filter(filter_func_preprocess)

In [ ]:
dva_vw, cap_vw = dva_extractor.get_dva(time=df_vw["time_h"], capacity=df_vw["Q"], voltage=df_vw["U"],crop_cap_Ah=(0,180))

In [ ]:
# the cell data is measured from 2.8-4.2V
# the vehicle data on cell level ranges from approx 3.3V-4.19V
# to account for this the vehicle data is shifted
deltaQ = dva_extractor.get_Q_shift_for_lower_cut_off_voltage(df_vw,df,n_seriell=108,n_parallel=2)

## Plots C/50

In [ ]:
def pOCV_plot(axes,df_FC,df_C,df_A,df_vw):
    sec_ax = axes.twinx()
    axes.set_zorder(sec_ax.get_zorder()-1)  # default zorder is 0 for ax1 and ax2
    axes.patch.set_visible(False)  # prevents ax1 from hiding ax2
    # offset for compact fig
    s=108
    p=2
    offset_interrup_y = s
    # vehicle data 
    axes.plot(df_vw["Q"],df_vw["U"]-offset_interrup_y,color="black")
    
    # labels
    axes.set_ylabel(r"Vehicle voltage in V",fontsize=7)
    #axes.set_xlabel(r"Cell charge in Ah",fontsize=7)
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=False)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.set_ylim([0,4.5*s-offset_interrup_y])
    axes.set_xlim([0,150])
    
    tick_positions = np.array([0,1,1.5,2.5,3.5])*s
    axes.set_yticks(tick_positions)
    axes.yaxis.set_minor_locator(FixedLocator(np.array([0.5,2,3])*s))
    
    axes.set_yticklabels([f"{label*s:.0f}" for label in [0,1,2.5,3.5,4.5]])
    tick_positions = np.array([0,50,100,150])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    axes.tick_params(axis='x', labelsize=7)
    axes.tick_params(axis='y', labelsize=7)
    axes.grid()
    axes.set_axisbelow(True)

    ## secondary axes
    # cell data
    sec_ax.plot(df["Q"]*p-deltaQ,df["U"]-1,color="black",linestyle="--")
    sec_ax.plot((df["Q"]-5)*p*1.08-deltaQ,df_C["U"]-1,color=VisualizationConfig.TUMorange.value)
    cap_A = df["Q"].values*0.93+0.5
    cap_A[cap_A>60]*=1.07
    sec_ax.plot(cap_A*p-deltaQ,df_A["U"],color=VisualizationConfig.TUMblau.value)

    sec_ax.set_ylim([0,4.5-1])
    tick_labels = np.array([0,1,1.5,2.5,3.5])
    sec_ax.set_yticks(tick_labels)
    sec_ax.set_yticklabels(["0","1.5","2.5","3.5","4.5"])
    sec_ax.tick_params(axis='y', labelsize=7)
    sec_ax.set_ylabel(r"Cell voltage in V",fontsize=7)
        
    twiny = axes.twiny()
    twiny.set_xlabel(r"Vehicle energy in kWh",fontsize=7) 
    twiny.set_xlim(0,150)
    tick_positions = np.array([0,50,100,150])
    twiny.set_xticks(tick_positions)
    twiny.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    twiny.tick_params(axis='x', labelsize=7)
    U_mean = df_vw["U"].mean()
    twiny.set_xticklabels([f"{round(label*U_mean/10000)*10:.0f}" for label in tick_positions])
  

    for k, spine in twiny.spines.items():  #ax.spines is a dictionary
        spine.set_zorder(1)
    # twiny.annotate(text="       ",
    #                       xy=(-2,136),
    #                       horizontalalignment="left",
    #                       verticalalignment="center",
    #                       fontsize=10,color="k",rotation=0,annotation_clip=False,
    #                 bbox=dict(boxstyle='square,pad=.0',fc='white', ec='white', alpha=1),zorder=2)   

    twiny.annotate(text="$\Vert$",
                          xy=(0,135),
                          horizontalalignment="center",
                          verticalalignment="center",
                          fontsize=11,color="k",rotation=90,zorder=3,
                  bbox=dict(boxstyle='square,pad=.001',fc='white', ec='None', alpha=1))

    twiny.annotate(text="$\Vert$",
                          xy=(150,135),
                          horizontalalignment="center",
                          verticalalignment="center",
                          fontsize=11,color="k",rotation=90,zorder=3,
                  bbox=dict(boxstyle='square,pad=.001',fc='white', ec='None', alpha=1))
    return axes

In [ ]:
def add_FOI_POCV(axes):
    height=200
    #axes.axvline(x=0, ymin=0, ymax=1,color=VisualizationConfig.TUMgrau50.value, linestyle="--",zorder=1)
    axes.annotate(text='', xy=(0,height), xytext=(150,height), arrowprops=dict(arrowstyle='<|-|>', shrinkA=0, shrinkB=0,facecolor="k"))
    axes.annotate(text=r'$Q_{calc.} = \int I \cdot dt$', xy=(150/2-20,height+2),horizontalalignment="left",verticalalignment="bottom",fontsize=7)
    axes.annotate(text=r'$E_{calc.} = \int UI \cdot dt$', xy=(150/2-20,height-6),horizontalalignment="left",verticalalignment="top",fontsize=7)
    return axes

In [ ]:
def DVA_plot(axes,cap,dva,cap_C,dva_C,cap_A,dva_A,cap_vw,dva_vw,deltaQ):
    sec_ax = axes.twinx()
    #axes.set_zorder(1) 
    #axes.set_frame_on(False)
    axes.set_zorder(sec_ax.get_zorder()-1)  # default zorder is 0 for ax1 and ax2
    #axes.set_frame_on(False)
    axes.patch.set_visible(False)  # prevents ax1 from hiding ax2
    # offset for compact fig
    Q_nom = np.max(cap_vw)
    s=108
    p=2
    offset_interrup_y = s
    # vehicle data 
    axes.plot(cap_vw,dva_vw*Q_nom,color="black")
    
    # labels
    axes.set_ylabel(r"$Q_{N,pack}\ \cdot \ \left | \frac{dU_{pack}}{dQ_{pack}} \right |$ in V",fontsize=7)
    axes.set_xlabel(r"Vehicle charge in Ah",fontsize=7) 
    axes.tick_params('x',which="both", top=True, labeltop=False,bottom=True, labelbottom=True)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.set_xlim(0,150)
    tick_positions = np.array([0,50,100,150])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    axes.tick_params(axis='x', labelsize=7)
    axes.tick_params(axis='y', labelsize=7)
    axes.grid()
    axes.set_axisbelow(True)
    
    axes.set_ylim([0,s*p])
    tick_positions = np.array([0,s,s*p])
    axes.set_yticks(tick_positions)
    axes.yaxis.set_minor_locator(FixedLocator([s/2,s+s/2]))
    
    # cell data
    sec_ax.plot(cap*2-deltaQ,dva*Q_nom/2,color="black",linestyle="--")
    sec_ax.plot(cap_C*2-deltaQ,dva_C*Q_nom/2,color=VisualizationConfig.TUMorange.value)
    sec_ax.plot(cap_A*2-deltaQ,dva_A*Q_nom/2,color=VisualizationConfig.TUMblau.value)
    sec_ax.set_ylim([0,2])
    sec_ax.set_ylabel(r"$Q_{N,cell}\ \cdot \ \left | \frac{dU_{cell}}{dQ_{cell}} \right |$ in V",fontsize=7)
    tick_positions = np.array([0,1,2])
    sec_ax.set_yticks(tick_positions)
    sec_ax.yaxis.set_minor_locator(FixedLocator([0.5,1.5]))
    return axes

In [ ]:
def add_FOI_DVA(axes):
    #define new axis on highest level
    newax = axes.twiny().twinx()
    newax.set_zorder(2)
    newax.set_ylim([0,216])
    newax.set_xlim([0,150])
    newax.get_xaxis().set_ticks([])
    newax.get_yaxis().set_ticks([])
    # define labels
    x_QB = 86.5
    height_NE = 190 
    #anode
    newax.axvline(x=x_QB, ymin=0.64, ymax=0.89,zorder=2, clip_on=False,color="k",linestyle="-",linewidth=1)
    newax.annotate(text='', xy=(0,height_NE), xytext=(x_QB,height_NE), arrowprops=dict(arrowstyle='<|-|>', shrinkA=0, shrinkB=0,facecolor="k"))
    newax.annotate(text=r'$Q_{NE}$', xy=((x_QB-0)/2,height_NE+2),horizontalalignment="center",verticalalignment="bottom",fontsize=7)
    # Balancing
    newax.annotate(text='', xy=(150,height_NE), xytext=(x_QB+0.4,height_NE), arrowprops=dict(arrowstyle='<|-|>', shrinkA=0, shrinkB=0,facecolor="k"))
    newax.annotate(text=r'$Q_B$', xy=((150-x_QB)/2+x_QB,height_NE+2),horizontalalignment="center",verticalalignment="bottom",fontsize=7)
    # #cathode
    x_QC = 134
    height_PE = 160
    newax.axvline(x=x_QC, ymin=0.58, ymax=0.75,zorder=2, clip_on=False,color="k",linestyle="-",linewidth=1)
    newax.annotate(text='', xy=(150,height_PE), xytext=(x_QC+0.4,height_PE), arrowprops=dict(arrowstyle='<|-|>', shrinkA=0, shrinkB=0,facecolor="k"))
    newax.annotate(text=r'$Q_{PE}$', xy=((150-x_QC)/2+x_QC,height_PE+3),horizontalalignment="center",verticalalignment="bottom",fontsize=7)
    #stages
    newax.annotate(text='4L', xy=(12.0*2-deltaQ,0.74*108), xytext=(7*2-deltaQ,1.15*108),fontsize=7,verticalalignment="bottom",
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMblau.value),color=VisualizationConfig.TUMblau.value)
    newax.annotate(text='3L', xy=(15.3*2-deltaQ,0.63*108), xytext=(17*2-deltaQ,1.1*108),fontsize=7,verticalalignment="bottom",
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMblau.value),color=VisualizationConfig.TUMblau.value)
    newax.annotate(text='2L', xy=(19.2*2-deltaQ,0.53*108), xytext=(23*2-deltaQ,0.8*108),fontsize=7,
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMblau.value),color=VisualizationConfig.TUMblau.value)
    newax.annotate(text='stage 2', xy=(47*2-deltaQ,0.9*108), xytext=(30*2-deltaQ,1.1*108),fontsize=7,verticalalignment="bottom",
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMblau.value),color=VisualizationConfig.TUMblau.value)
                     
    newax.annotate(text='H1', xy=(7.8*2-deltaQ,0.33*108), xytext=(13*2-deltaQ,0.05*108),fontsize=7,color=VisualizationConfig.TUMorange.value,
                   arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMorange.value))
    
    newax.annotate(text='M', xy=(51.5*2-deltaQ,0.78*108), xytext=(54*2-deltaQ,0.24*108),fontsize=7,color=VisualizationConfig.TUMorange.value,
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMorange.value))
                      
    newax.annotate(text='H2', xy=(70.5*2-deltaQ,0.96*108), xytext=(72*2-deltaQ,0.54*108),fontsize=7,color=VisualizationConfig.TUMorange.value,
                      arrowprops=dict(arrowstyle='-', shrinkA=0, shrinkB=0, mutation_scale=20,color=VisualizationConfig.TUMorange.value))
    return axes

In [ ]:
nrow = 2
fig, axes= plt.subplots(nrow,1,
                        figsize=(cm2inch(VisualizationConfig.columnwidth_in_cm.value),
                                nrow*cm2inch(4.8)))
axes = axes.flatten()
axes[0] = pOCV_plot(axes[0],df,df_C,df_A,df_vw)
axes[1] = DVA_plot(axes[1],cap,dva,
                           cap_C_trafo,dva_C_trafo,
                           cap_A_trafo,dva_A_trafo,
                            cap_vw,dva_vw,deltaQ)

axes[0] = add_FOI_POCV(axes[0])
axes[1] = add_FOI_DVA(axes[1])
axes[0].yaxis.set_label_coords(-0.135, 0.5)
axes[1].yaxis.set_label_coords(-0.11, 0.5) 
# legend
handles, labels =axes[0].get_legend_handles_labels()  
line = plt.hlines(-1,0,1,label='vehicle ', color="black")
line1 = plt.hlines(-1,0,1,label='full cell', color="black",linestyle="--")
line2 = plt.hlines(-1,0,1,label='negative electrode', color=VisualizationConfig.TUMblau.value)
line3 = plt.hlines(-1,0,1,label='positive electrode', color=VisualizationConfig.TUMorange.value)

# add manual symbols to auto legend
handles.extend([line,line1, line2,line3])
axes[0].legend(handles=handles,loc="center",ncols=2,handletextpad=0.4,fontsize=7,
           framealpha =1 , fancybox=False,labelspacing=0.1,columnspacing=1,edgecolor="black",bbox_to_anchor=(0.5, 1.45),handlelength=1.8)


plt.tight_layout()
# NR
axes[0].text(-0.18, 1.1, r'a)', fontsize=7, horizontalalignment='center',
     verticalalignment='center', transform=axes[0].transAxes)
axes[1].text(-0.18, 1.1, r'b)', fontsize=7, horizontalalignment='center',
     verticalalignment='center', transform=axes[1].transAxes)

fig.savefig(os.path.join(GeneralConfig.path2figures.value,"00-POCV_DVA_C_NMC532_halfcells.pdf"))
fig.savefig(os.path.join(GeneralConfig.path2figures.value,"00-POCV_DVA_C_NMC532_halfcells.png"),dpi=300)